In [74]:
import os
import time
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary


In [ ]:

'''Para libros: https://www.amazon.es/dp/8434417219'''
ISBN10 = '8434417219' 
'''Para productos que llevan EAN, en informacion sobre el modelo '''
ean_list = ['8487497376498','B00NBVIMDM']


In [85]:
def bullet_points(all_bullets): 
    
    if len(all_bullets.split("\n")) == 8: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8= [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 7: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7= [str(e) for e in all_bullets.split("\n")] 
        bullet8 = float('nan')
    elif len(all_bullets.split("\n")) == 6: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6 = [str(e) for e in all_bullets.split("\n")] 
        bullet7 = bullet8 = float('nan')
    elif len(all_bullets.split("\n")) == 5: 
        bullet1, bullet2, bullet3, bullet4, bullet5 = [str(e) for e in all_bullets.split("\n")] 
        bullet6 = bullet7 = bullet8 =  float('nan')
    elif len(all_bullets.split("\n")) == 4: 
        bullet1, bullet2, bullet3, bullet4 = [str(e) for e in all_bullets.split("\n")] 
        bullet5 = bullet6 = bullet7 = bullet8 = float('nan')
    elif len(all_bullets.split("\n")) == 3: 
        bullet1, bullet2, bullet3, = [str(e) for e in all_bullets.split("\n")]
        bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan')
    elif len(all_bullets.split("\n")) == 2: 
        bullet1, bullet2 = [str(e) for e in all_bullets.split("\n")]
        bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan')
    elif len(all_bullets.split("\n")) == 1: 
        bullet1 = [str(e) for e in all_bullets.split("\n")]
        bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan')
    else : 
        bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan') 
        
    return bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8  



def amazon(ean_list):
    
    amazon_info = pd.DataFrame()
    Detalles_tecnicos_t = pd.DataFrame()
    Detalles_usuarios_amazon_t = pd.DataFrame()
    
    for ean in ean_list:
        
        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.amazon.es')

        search = browser.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//div[@class='sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'])[2]")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@id="title"]')[0].text
        price = browser.find_elements_by_xpath('//span[@id="priceblock_ourprice"]')[0].text
        all_bullets = browser.find_elements_by_xpath('//div[@id="feature-bullets"]')[0].text
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8 = bullet_points(all_bullets)
        main_image = browser.find_element_by_xpath('//img[@id="landingImage"]').get_attribute('src')
        
        amazon_selenium = pd.DataFrame({'ean.value': [ean],
                                     'item_name.value': [title], 
                                     'list_price.value_with_tax': [price],
                                     'bullet_point.value': [bullet1], 
                                     'bullet_point#2.value': [bullet2],
                                     'bullet_point#3.value': [bullet3], 
                                     'bullet_point#4.value': [bullet4], 
                                     'bullet_point#5.value': [bullet5],
                                     'bullet_point#6.value': [bullet6],
                                     'bullet_point#7.value': [bullet7],  
                                     'main_image': [main_image]
                                    })
        
        try:
            
            url = str(browser.current_url)
            time.sleep(10)
            lista = pd.read_html(url, match='.+', flavor = None, header = None, index_col =None, skiprows=None, attrs=None, parse_dates=False, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True, displayed_only=True)
            time.sleep(10)

            Detalles_tecnicos = lista[1]
            Detalles_usuarios_amazon = lista[2]

            Detalles_tecnicos.set_index(Detalles_tecnicos.columns[0])
            Detalles_tecnicos_t = Detalles_tecnicos.T
            Detalles_tecnicos_t.columns = Detalles_tecnicos_t.iloc[0]
            Detalles_tecnicos_t = Detalles_tecnicos_t.drop(Detalles_tecnicos_t.index[0]).reset_index(drop=True)

            Detalles_usuarios_amazon.set_index(Detalles_usuarios_amazon.columns[0])
            Detalles_usuarios_amazon_t = Detalles_usuarios_amazon.T
            Detalles_usuarios_amazon_t.columns = Detalles_usuarios_amazon_t.iloc[0]
            Detalles_usuarios_amazon_t = Detalles_usuarios_amazon_t.drop(Detalles_usuarios_amazon_t.index[0]).reset_index(drop=True)

        except urllib.error.HTTPError:
        
            browser.close()

            amazon_data = pd.concat([amazon_selenium, Detalles_tecnicos_t,Detalles_usuarios_amazon_t], axis=1, sort=False)
            amazon_data = amazon_data.set_index('ean.value')

            amazon_info = pd.concat([amazon_info, amazon_data], axis=1, sort=False)

        '''
        amazon_info= pd.concat([amazon_info, amazon_data])
        '''
    return amazon_info


In [89]:
df = amazon(ean_list)

KeyboardInterrupt: 

In [90]:
df

""
